# GSL PERIODOGRAMS - RV BATCH FILE PROCESSING

In [1]:
# NOTA IMPORTANTE - CÓMO IMPLEMENTAR LO DEL "TIMEOUT".
# https://stackoverflow.com/questions/25027122/break-the-function-after-certain-time
# NO SIRVE - SOLO SIRVE PARA ENTORNOS UNIX

This notebook takes the GTO objects table and calculates the Lomb-Scargle periodogram for each of the RV curves, storing the results in `FITS` files with a primary header and two additional header data units, one with the RV curve itself and another with the periodogram.

Additionally, the GTO objects table and file is updated with new columns storing the results for each object.

Error control is implemented to prevent exceptions and timeouts to interrupt the processing. In case any of these errors happen, it is so recorded in the results table. Timeout value is dinamically updated, as the calculation for each record is completed.

**NOTE:** there seems to be some kind of memory leak or something similar, because the browser crashes if you try to execute all loop iterations at once. Hence, it is necessary to execute the loop in batches of 100 elements, for example.

**Calculation conditions:**

The stars currently under analysis are in the $He_{3}$ instability band, so they are expected to have effective temperatures in the range $T_{eff}\in[3300, 4300] K$, $\log g\in[4.5, 5.1]$, and masses in the range $M_{star}\in[0.20, 0.60] M_{Sun}$.

According to _Table 2_ in _The theoretical instability strip of M dwarf stars (Rodríguez-López, C., et al. 2014, MNRAS, 438, 2371)_ these stars have typical periods of $20 min$ to $3 h$, so we will set the limits of the periodogram frequencies for periods between $5 min$ and $10 h$. In days, this corresponds to a range of $P\in[0.003472, 0.416667] d$ or, equivalently, frequencies in the range $f\in[288.0, 2.4] d^{-1}$.

We will use the _Generalized Lomb Scargle Periodogram_ method, as described in [Zechmeister and Kürster, 2009](https://www.aanda.org/articles/aa/full_html/2009/11/aa11296-08/aa11296-08.html) and implemented by GitHub repository [mzechmeister/GLS](https://github.com/mzechmeister/GLS), using the default _ZK_ normalization.

## Modules and configuration

### Modules

In [2]:
# Modules import:
#from collections import OrderedDict
import pandas as pd
import numpy as np
import time

import warnings

from scipy import stats

# https://github.com/mzechmeister/GLS
from gls import Gls

from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
from astropy import units as u
from astropy.io import fits
#from astropy.time import Time

import lightkurve as lk

#%matplotlib inline
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [3]:
# Configuration:
# Files and folders (WARNING: THIS FOLDER STRUCTURE MUST EXIST PREVIOUSLY):
GTO_FILE = "../data/CARM_VIS_objects_with_PG.csv" # NOTE: initially this should be a copy of the previous file.
IN_RV_FOLDER = "../data/CARM_VIS_RVs/"
OUT_IMG_FOLDER = "../data/CARM_VIS_RVs_PGs/figures/"
OUT_PROCESSED_FOLDER = "../data/CARM_VIS_RVs_PGs/"
#OUT_GTO_FILE = "../data/GTO_objects_withRVPG.csv"
#OUT_PG_FILE = "../data/GTO_PGs.csv"
#OUT_ERROR_FILE = "../data/GTO_PG_ERRORs.csv"

# Periodogram constants:
FBEG = 2.4 # d^{-1}, corresponds to a period P=10 hours
FEND = 288 # d^{-1}, corresponds to a period P=5 min
NUM_POINTS = 2856001 # 2856000 for RV time-series
PBEG = None # Default
PEND = None # Default
OFAC = 10 # Default
HIFAC = 1 # Default
#FREQ = np.linspace(start=FBEG, stop=FEND, num=1000001) # 1,000,001 points between 4.8 and 144 d^(-1)
FREQ = np.linspace(start=FBEG, stop=FEND, num=NUM_POINTS)
# Must be compatible with FBEG, FEND values
NORM = "ZK" # Default
LS = False # Default
FAST = False # Default
VERBOSE = False # Default
FAP_LEVELS_PLOT = [0.01, 0.05, 0.10] # FAP reference levels to plot

# PULSATION RANGE OF INTEREST (FOR THE PLOTS):
F_LOW = 8 # P=0.1250 d (3 h)
F_HIGH = 72 # P=0.0139 d (20 min)

# Initial timeout value (to prevent a stuck file to interrupt the whole process)
#INITIAL_TIMEOUT = 600 # Seconds


In [4]:
def rv_load(filename: str):
    '''Load the RV file and returns a lightcurve object'''
    rv_lk = Table.read(filename, format='ascii',
                    names = ['time', 'RV', 'eRV'], units=[u.d, u.meter / u.second, u.meter / u.second])
    rv_lk = lk.LightCurve(time=rv_lk['time'], flux=rv_lk['RV'], flux_err=rv_lk['eRV'])
    return rv_lk

In [5]:
def rv_infer_sampling(rv_lk: lk.LightCurve):
    '''Infer sampling period from light curve'''
    time_diffs = rv_lk['time'][1:] - rv_lk['time'][:-1]
    return np.median(time_diffs)

## Data processing

### GTO data loading

In [6]:
# Load GTO data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_T0_PG_TESS,WF_e_T0_PG_TESS,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000997,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.689208,0.027575,999.999756,2.151008e-06,1.0,1.0,NaN,94.758838,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23548+38...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1353.926624,0.036481,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN


In [7]:
gto.tail(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_T0_PG_TESS,WF_e_T0_PG_TESS,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS
385,J00183+440,GX And,A,15A,4.613226,44.024787,00:18:22.88,+44:01:22.6,116.690592,-18.446865,...,1764.644951,0.03086,999.999756,0.000002,1.0,1.0,NaN,191.241441,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00183+44...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
386,J00162+198W,EZ Psc,Aab,1006A,4.064321,19.857041,00:16:14.63,+19:51:37.5,111.729451,-42.248173,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN
387,J00162+198E,LP 404-062,B,1006B,4.070651,19.860692,00:16:16.15,+19:51:50.5,111.738263,-42.245679,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN
388,J00067-075,GJ 1002,-,1002,1.676350,-7.546475,00:06:43.20,-07:32:17.0,92.444693,-67.730511,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN
389,J00051+457,GJ 2,C,2,1.300921,45.785894,00:05:10.89,+45:47:11.6,114.559789,-16.332933,...,1764.644774,0.03086,999.999756,0.000002,1.0,1.0,NaN,187.918468,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00051+45...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...


In [8]:
gto.shape

(390, 298)

In [9]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

Generate the proper auxiliary columns (with the basic periodograms results).

In [10]:
# Additional columns:
if 'n_RV' in gto.columns:
    pass
else:
    gto['n_RV'] = None # Number of points in RV curve
    gto['Ps_RV'] = None # Sampling period (d)
    gto['fs_RV'] = None # Sampling frequency (d^(-1))
    gto['wmean_RV'] = None # Mean of RV
    gto['wrms_RV'] = None # RMS of RV
    gto['info_PG_RV'] = None # Information text about the PG
    gto['maxP_PG_RV'] = None # Max power value in the PG
    gto['maxSNR_PG_RV'] = None # Max power value in the PG
    gto['rms_PG_RV'] = None # RMS value in the PG residuals
    gto['f_PG_RV'] = None # Best frequency in the PG (d^(-1))
    gto['e_f_PG_RV'] = None # Error of the best frequency in the PG (d^(-1))
    gto['Pd_PG_RV'] = None # Best period in the PG (d)
    gto['e_Pd_PG_RV'] = None # Error of the best period in the PG (d)
    gto['Ph_PG_RV'] = None # Best period in the PG (hours)
    gto['e_Ph_PG_RV'] = None # Error of the best period in the PG (hours)
    gto['Pm_PG_RV'] = None # Best period in the PG (minutes)
    gto['e_Pm_PG_RV'] = None # Error of the best period in the PG (minutes)
    gto['A_PG_RV'] = None # Amplitude of the best frequency
    gto['e_A_PG_RV'] = None # Error of the amplitude of the best frequency
    gto['ph_PG_RV'] = None # Amplitude of the best frequency
    gto['e_ph_PG_RV'] = None # Error of the amplitude of the best frequency
    gto['T0_PG_RV'] = None # Reference epoch of the best frequency
    gto['e_T0_PG_RV'] = None # Error of the epoch of the best frequency
    gto['offset_PG_RV'] = None # Offset of the best frequency
    gto['e_offset_PG_RV'] = None # Error of the offset of the best frequency
    gto['FAP_PG_RV'] = None # False alarm probability
    gto['valid_PG_RV'] = None # Flag to indicate if the periodogram calculation succeeded (1) or not (0).
    gto['error_PG_RV'] = None # The error raised during processing. Empty if processing was successful.
    gto['elapsed_time_PG_RV'] = None # The time elapsed in calculation
    gto['fits_file_RV'] = None # The name of the processed file.
    gto['fig_file_RV'] = None # The name of the figure file.


### Batch processing of all RV files

In [11]:
n = len(gto)
n

390

#### Batch processing

**Note:** previous data (not NZP-corrected) failed on indexes: 99, 273 and 344.

In [12]:
warnings.filterwarnings('ignore')
# Batch processing:
lapse_list = []
median_lapse = None
# NOTE THE FOR LOOP IS SEPARATED INTO SEVERAL "BATCHES", SO AS TO PREVENT POTENTIAL MEMORY PROBLEMS
# OR TOO MUCH TIME ELAPSED
#for i in range(0, 3): # TEST
#for i in [387, 388, 389]: # To repeat the calculation for specific objects.
#for i in range(0, len(gto)): # ALL RECORDS
#for i in range(0, 96):
#for i in range(96, 192):
#for i in range(192, 290):
for i in range(290, len(gto)):
    start_time = time.time()
    # Names:
    karmn = gto.loc[i, 'Karmn'] # KARMENES name
    commn = gto.loc[i, 'Name'] # Common name
    tic_id = str(gto.loc[i, 'TIC_id']) # TESS TIC identifier
    print("Record: %d, started at %s"
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    print("Processing %s star..." %karmn)
    try:
        # LOAD RV FILE:
        rv_file = gto.loc[i, 'rv_file']
        print("filename: %s" %rv_file)
        rv_lk = rv_load(rv_file)
        
        # GENERATE PERIODOGRAM:
        gls = Gls((rv_lk['time'].value, rv_lk['flux'].value, rv_lk['flux_err'].value),
              fbeg=FBEG, fend=FEND, Pbeg=PBEG, Pend=PEND, ofac=OFAC, hifac=HIFAC, freq=FREQ,
              norm=NORM, ls=LS, fast=FAST, verbose=VERBOSE)
        
        # GENERATE THE FITS FILE:
        # Prepare the fits primary HDU (only header):
        primary_header = fits.Header()
        primary_header['OBJECT'] = (karmn, "KARMENES target name")
        primary_header['NAME'] = (commn, "Object common name")
        primary_header['TIC'] = (tic_id, "Object TESS identifier")
        primary_header['RA_J2000'] = ("00:05:10.89", "Object right ascension (J2000)")
        primary_header['DE_J2000'] = ("+45:47:11.6", "Object declination (J2000)")
        primary_header['SPTYPE'] = ("M1.0 V", "Spectral type")
        primary_header['TEFF_K'] = (3773, "Effective temperature in Kelvin")
        primary_header['LOGG'] = (5.07, "Logarithm of surface gravity")
        primary_header['FEH'] = (-0.04, "Metallicity")
        primary_header['L_LSUN'] = (0.0436229, "Luminosity in Solar luminosities")
        primary_header['R_RSUN'] = (0.48881, "Radius in Solar radii")
        primary_header['M_MSUN'] = (0.4918, "Mass in Solar masses")
        primary_header['D_PC'] = (11.50352803, "Distance in parsec")
        hdu_primary = fits.PrimaryHDU(header=primary_header)

        # Prepare the RV HDU:
        hdu_rv = fits.table_to_hdu(QTable(rv_lk.to_table()))
        hdu_rv.name = "RV_CURVE"
        psample = rv_infer_sampling(rv_lk).value
        fsample = 1.0 / psample
        fnyq = 2.0 * fsample
        freq_units = u.d ** (-1)
        hdu_rv.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_rv.header['PUNIT'] = u.d.to_string(format='fits')
        hdu_rv.header['FUNIT'] = freq_units.to_string(format='fits')
        hdu_rv.header['RVPOINTS'] = (gls.N, "Number of points in the RV curve")
        hdu_rv.header['AVGFLUX'] = (gls._Y, "Average flux of RV curve")
        hdu_rv.header['RMSFLUX'] = (np.sqrt(gls._YY), "Flux RMS of RV curve")
        hdu_rv.header['PSAMPLE'] = (psample, "Inferred cadence in RV curve")
        hdu_rv.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in RV curve")
        hdu_rv.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
                
        # Prepare the PG HDU:
        hdu_pg = fits.table_to_hdu(
            QTable(data=[gls.freq, gls.power], names=['freq', 'power'], 
                   units=[1.0 / u.d, (u.m / u.s) ** 2]))
        hdu_pg.name = "GLS_PG"
        fpoints = len(gls.f)
        fres = (gls.fend - gls.fbeg) / (fpoints - 1)
        hdu_pg.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_pg.header['FUNIT'] = (freq_units.to_string(format='fits'), "Unit for frequencies")
        hdu_pg.header['PUNIT'] = (u.d.to_string(format='fits'), "Unit for periods")
        hdu_pg.header['PK_FREQ'] = (gls.best['f'], "Frequency of the peak in periodogram")
        hdu_pg.header['PK_POW'] = (gls.pmax, "Power of the peak in periodogram")
        hdu_pg.header['PK_SNR'] = (gls.best['amp'] / gls.rms, "SNR of the peak in periodogram")
        hdu_pg.header['PK_FAP'] = (gls.FAP(Pn=None), "FAP of the peak in periodogram")
        hdu_pg.header['RES_RMS'] = (gls.rms, "RMS of residuals in periodogram")
        hdu_pg.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in RV curve")
        hdu_pg.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
        hdu_pg.header['FPOINTS'] = (fpoints, "Number of points in periodogram")
        hdu_pg.header['FBEG'] = (gls.fbeg, "Start frequency in periodogram")
        hdu_pg.header['FEND'] = (gls.fend, "End frequency in periodogram")
        hdu_pg.header['FRES'] = (fres, "Frequency resolution in periodogram")
        hdu_pg.header['F'] = (gls.best['f'], "Peak best estimate: frequency")
        hdu_pg.header['E_F'] = (gls.best['e_f'], "Peak best estimate: frequency error")
        hdu_pg.header['P'] = (gls.best['P'], "Peak best estimate: period")
        hdu_pg.header['E_P'] = (gls.best['e_P'], "Peak best estimate: period error")
        hdu_pg.header['A'] = (gls.best['amp'], "Peak best estimate: amplitude")
        hdu_pg.header['E_A'] = (gls.best['e_amp'], "Peak best estimate: amplitude error")
        hdu_pg.header['PH'] = (gls.best['ph'], "Peak best estimate: phase")
        hdu_pg.header['E_PH'] = (gls.best['e_ph'], "Peak best estimate: phase error")
        hdu_pg.header['T0'] = (gls.best['T0'], "Peak best estimate: frequency")
        hdu_pg.header['E_T0'] = (gls.best['e_T0'], "Peak best estimate: frequency error")
        hdu_pg.header['OFF'] = (gls.best['offset'], "Peak best estimate: offset")
        hdu_pg.header['E_OFF'] = (gls.best['e_offset'], "Peak best estimate: offset error")
        hdu_pg.header['OFAC'] = (gls.ofac, "Setup: oversampling factor")
        hdu_pg.header['HIFAC'] = (gls.ofac, "Setup: maximum frequency factor")
        hdu_pg.header['NORM'] = (gls.norm, "Setup: normalization type")
        hdu_pg.header['LS'] = (gls.ls, "Setup: conventional Lomb-Scargle calculation")
        hdu_pg.header['FAST'] = (gls.fast, "Setup: fast evaluation, recursive trigonometric")
        
        # Create and save the fits file:
        fits_filename = OUT_PROCESSED_FOLDER + karmn + "_RV_PG.fits"
        hdul = fits.HDUList([hdu_primary, hdu_rv, hdu_pg])
        hdul.writeto(fits_filename, overwrite=True)
        
        # Delete all the HDUs:
        del hdul
        del hdu_primary
        del hdu_rv
        del hdu_pg

        # FILL IN THE DATA IN THE GTO TABLE:
        gto.loc[i, 'n_RV'] = gls.N
        gto.loc[i, 'Ps_RV'] = psample
        gto.loc[i, 'fs_RV'] = fsample
        gto.loc[i, 'wmean_RV'] = gls._Y
        gto.loc[i, 'wrms_RV'] = np.sqrt(gls._YY)
        gto.loc[i, 'info_PG_RV'] = gls.info(stdout=False)
        gto.loc[i, 'maxP_PG_RV'] = gls.power.max()
        gto.loc[i, 'maxSNR_PG_RV'] = gls.best['amp'] / gls.rms
        gto.loc[i, 'rms_PG_RV'] = gls.rms
        gto.loc[i, 'f_PG_RV'] = gls.best['f']
        gto.loc[i, 'e_f_PG_RV'] = gls.best['e_f']
        gto.loc[i, 'Pd_PG_RV'] = gls.best['P']
        gto.loc[i, 'e_Pd_PG_RV'] = gls.best['e_P']
        gto.loc[i, 'Ph_PG_RV'] = 24.0 * gls.best['P']
        gto.loc[i, 'e_Ph_PG_RV'] = 24.0 * gls.best['e_P']
        gto.loc[i, 'Pm_PG_RV'] = 24.0 * 60.0 * gls.best['P']
        gto.loc[i, 'e_Pm_PG_RV'] = 24.0 * 60.0 * gls.best['e_P']
        gto.loc[i, 'A_PG_RV'] = gls.best['amp']
        gto.loc[i, 'e_A_PG_RV'] = gls.best['e_amp']
        gto.loc[i, 'ph_PG_RV'] = gls.best['ph']
        gto.loc[i, 'e_ph_PG_RV'] = gls.best['e_ph']
        gto.loc[i, 'T0_PG_RV'] = gls.best['T0']
        gto.loc[i, 'e_T0_PG_RV'] = gls.best['e_T0']
        gto.loc[i, 'offset_PG_RV'] = gls.best['offset']
        gto.loc[i, 'e_offset_PG_RV'] = gls.best['e_offset']
        gto.loc[i, 'FAP_PG_RV'] = gls.FAP(Pn=None)
        
        # GENERATE THE FIGURE:
        fig = gls.plot(block=False, period=False,
                       fap=FAP_LEVELS_PLOT, gls=True, data=True, residuals=True)
        # Add the reference lines for predicted pulsations:
        fig.axes[0].axvline(F_LOW, color="darkgray", linestyle="--")
        fig.axes[0].axvline(F_HIGH, color="darkgray", linestyle="--")
        figure_title = "%s, P=%.4f [min], FAP=%.4f%%" \
            %(karmn, gto.loc[i, 'Pm_PG_RV'], 100.0 * gto.loc[i, 'FAP_PG_RV'])
        fig.suptitle(figure_title, fontdict = {'fontsize' : 36})
        fig.tight_layout()
        # Save the figure to disk:
        fig_file = OUT_IMG_FOLDER + karmn + "_RV_PG.jpg"
        fig.savefig(fig_file)
        plt.close() # Prevent the figure from showing.

        # SET THE RECORD CALCULATION AS VALID AND STORE THE REULTING FILENAMES
        gto.loc[i, 'valid_PG_RV'] = 1
        gto.loc[i, 'error_PG_RV'] = ""
        gto.loc[i, 'fits_file_RV'] = fits_filename
        gto.loc[i, 'fig_file_RV'] = fig_file

        # UPDATE THE AVERAGE RECORD PROCESSING TIME:
        lapse = time.time() - start_time
        lapse_list.append(lapse)
        median_lapse = np.nanmedian(lapse_list)
        gto.loc[i, 'elapsed_time_PG_RV'] = lapse
        
        # SAVE THE UPDATED GTO TABLE TO DISK:
        gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        
        # Report successful execution:
        print("Elapsed time: %.2f seconds" %lapse)
        print("... SUCCESS.")
        
        # Clear memory (delete the 'gls' object'):
        try:
            del gls
        except:
            pass
        
    except Exception as e:
        # Some error happened, establish the record as not valid and record the error:
        error = "*** Some ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, e)
        print(error)
        gto.loc[i, 'valid_PG_RV'] = 0
        gto.loc[i, 'error_PG_RV'] = e
        
        # Try to update the record, and save the file:
        try:
            # UPDATE THE AVERAGE RECORD PROCESSING TIME:
            lapse = time.time() - start_time
            lapse_list.append(lapse)
            median_lapse = np.nanmedian(lapse_list)
            gto.loc[i, 'elapsed_time_PG_RV'] = lapse
            print("Elapsed time: %.2f seconds" %lapse)

            # SAVE THE UPDATED GTO TABLE TO DISK:
            gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        except Exception as e2:
            error_2 = "*** Additional ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, str(e2))
            print(error)
            gto.loc[i, 'error_PG_RV'] = gto.loc[i, 'error_PG_RV'] + "/" + str(e2)

        # Clear memory (delete the 'gls' object'):
        try:
            del gls
        except:
            pass


Record: 290, started at 19/08/2022, 11:49:59
Previous median lapse time: None
Processing J06396-210 star...
filename: ../data/CARM_VIS_RVs/J06396-210.avc.dat
Elapsed time: 32.40 seconds
... SUCCESS.
Record: 291, started at 19/08/2022, 11:50:32
Previous median lapse time: 32.40 seconds
Processing J06371+175 star...
filename: ../data/CARM_VIS_RVs/J06371+175.avc.dat
Elapsed time: 39.07 seconds
... SUCCESS.
Record: 292, started at 19/08/2022, 11:51:11
Previous median lapse time: 35.74 seconds
Processing J06318+414 star...
filename: ../data/CARM_VIS_RVs/J06318+414.avc.dat
Elapsed time: 34.08 seconds
... SUCCESS.
Record: 293, started at 19/08/2022, 11:51:45
Previous median lapse time: 34.08 seconds
Processing J06246+234 star...
filename: ../data/CARM_VIS_RVs/J06246+234.avc.dat
Elapsed time: 33.26 seconds
... SUCCESS.
Record: 294, started at 19/08/2022, 11:52:18
Previous median lapse time: 33.67 seconds
Processing J06105-218 star...
filename: ../data/CARM_VIS_RVs/J06105-218.avc.dat
Elapsed ti

Elapsed time: 29.18 seconds
... SUCCESS.
Record: 330, started at 19/08/2022, 12:12:25
Previous median lapse time: 32.98 seconds
Processing J04198+425 star...
filename: ../data/CARM_VIS_RVs/J04198+425.avc.dat
Elapsed time: 31.27 seconds
... SUCCESS.
Record: 331, started at 19/08/2022, 12:12:56
Previous median lapse time: 32.97 seconds
Processing J04173+088 star...
filename: ../data/CARM_VIS_RVs/J04173+088.avc.dat
*** Some ERROR happened with record #331, J04173+088 star. Error=zero-size array to reduction operation minimum which has no identity
Elapsed time: 0.01 seconds
Record: 332, started at 19/08/2022, 12:12:56
Previous median lapse time: 32.93 seconds
Processing J04167-120 star...
filename: ../data/CARM_VIS_RVs/J04167-120.avc.dat
Elapsed time: 30.55 seconds
... SUCCESS.
Record: 333, started at 19/08/2022, 12:13:27
Previous median lapse time: 32.88 seconds
Processing J04153-076 star...
filename: ../data/CARM_VIS_RVs/J04153-076.avc.dat
Elapsed time: 32.06 seconds
... SUCCESS.
Record:

Elapsed time: 34.76 seconds
... SUCCESS.
Record: 369, started at 19/08/2022, 12:33:07
Previous median lapse time: 32.50 seconds
Processing J01352-072 star...
filename: ../data/CARM_VIS_RVs/J01352-072.avc.dat
Elapsed time: 30.01 seconds
... SUCCESS.
Record: 370, started at 19/08/2022, 12:33:37
Previous median lapse time: 32.46 seconds
Processing J01339-176 star...
filename: ../data/CARM_VIS_RVs/J01339-176.avc.dat
Elapsed time: 32.77 seconds
... SUCCESS.
Record: 371, started at 19/08/2022, 12:34:10
Previous median lapse time: 32.50 seconds
Processing J01125-169 star...
filename: ../data/CARM_VIS_RVs/J01125-169.avc.dat
Elapsed time: 38.73 seconds
... SUCCESS.
Record: 372, started at 19/08/2022, 12:34:49
Previous median lapse time: 32.63 seconds
Processing J01066+192 star...
filename: ../data/CARM_VIS_RVs/J01066+192.avc.dat
Elapsed time: 34.69 seconds
... SUCCESS.
Record: 373, started at 19/08/2022, 12:35:23
Previous median lapse time: 32.77 seconds
Processing J01056+284 star...
filename: 

## Check the calculations that took longer to complete

We now check the calcultions that took longer, so as to try to repeat them (as it could be due to a problem with the computer itself - for example, the program stopped when the computer went into "sleep" mode).

In [13]:
gto[['Karmn', 'elapsed_time_PG_RV']].sort_values(by='elapsed_time_PG_RV', ascending=False) \
    .head(10)

,Karmn,elapsed_time_PG_RV
99,J17578+046,72.310163
281,J07274+052,70.697308
14,J22565+165,66.454178
207,J11033+359,59.951127
163,J13299+102,58.380259
288,J06548+332,55.920416
4,J23492+024,54.071770
247,J09144+526,49.766129
160,J13457+148,49.013097
94,J18075-159,48.322945


## Review a given file

In [14]:
object_name = "J23556-061"

In [15]:
read_hdul = fits.open(OUT_PROCESSED_FOLDER + object_name + "_RV_PG.fits", mode='update', memmap=False)
read_hdul

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x00000187A0993AC0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001879902A370>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001879F2224F0>]

In [16]:
read_hdul.info()

Filename: ../data/CARM_VIS_RVs_PGs/J23556-061_RV_PG.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      17   ()      
  1  RV_CURVE      1 BinTableHDU     33   30R x 3C   [2D, D, D]   
  2  GLS_PG        1 BinTableHDU     46   2856001R x 2C   [D, D]   


### Retrieve basic information

In [17]:
read_hdul['PRIMARY'].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
OBJECT  = 'J23556-061'         / KARMENES target name                           
NAME    = 'GJ 912  '           / Object common name                             
TIC     = '9759680 '           / Object TESS identifier                         
RA_J2000= '00:05:10.89'        / Object right ascension (J2000)                 
DE_J2000= '+45:47:11.6'        / Object declination (J2000)                     
SPTYPE  = 'M1.0 V  '           / Spectral type                                  
TEFF_K  =                 3773 / Effective temperature in Kelvin                
LOGG    =                 5.07 / Logarithm of surface gravity                   
FEH     =                -0.

### Retrieve RV curve information

In [18]:
read_hdul['RV_CURVE']

In [19]:
read_hdul['RV_CURVE'].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   32 / length of dimension 1                          
NAXIS2  =                   30 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    3 / number of table fields                         
TIMESYS = 'UTC     '           / Default time scale                             
JDREF   =                  0.0 / Time columns are jd = jd1 + jd2                
TREFPOS = 'TOPOCENTER'         / Time reference position                        
TTYPE1  = 'time    '                                                            
TFORM1  = '2D      '        

In [20]:
read_hdul['RV_CURVE'].data[:10]

FITS_rec([([ 2.45759400e+06, -3.41206829e-01], 368.57269438, 2.11976458),
          ([ 2.45759600e+06, -3.31648916e-01], 363.74366739, 1.92925593),
          ([ 2.45760500e+06, -3.59053629e-01], 363.29476099, 2.43537069),
          ([ 2.45760900e+06, -3.55222573e-01], 363.44968482, 2.071301  ),
          ([ 2.45762600e+06, -3.96350565e-01], 357.61959182, 2.28650246),
          ([ 2.45763500e+06, -4.27978922e-01], 345.47360276, 2.72104472),
          ([ 2.45764500e+06, -4.58771974e-01], 343.73727152, 2.40985396),
          ([ 2.45767300e+06, -4.85209939e-01], 338.86742184, 1.77064468),
          ([ 2.45775300e+06,  2.83939595e-01], 308.38480361, 2.24009984),
          ([ 2.45793200e+06, -3.60266469e-01], 218.29813027, 2.16691784)],
         dtype=(numpy.record, [('time', '>f8', (2,)), ('flux', '>f8'), ('flux_err', '>f8')]))

In [21]:
read_rv_table = QTable.read(read_hdul['RV_CURVE'])
read_rv_table

time [2],flux,flux_err
d,m / s,m / s
float64,float64,float64
2457594.0 .. -0.34120682859793305,368.57269437885606,2.1197645791859014
2457596.0 .. -0.33164891554042697,363.7436673901377,1.9292559320919738
2457605.0 .. -0.35905362851917744,363.2947609879404,2.4353706850341275
2457609.0 .. -0.3552225725725293,363.44968482330813,2.071301001326418
2457626.0 .. -0.39635056536644697,357.6195918175518,2.2865024646363756
2457635.0 .. -0.4279789221473038,345.47360275795796,2.7210447228094123
2457645.0 .. -0.4587719738483429,343.7372715200439,2.4098539557767973
2457673.0 .. -0.48520993907004595,338.86742184033426,1.770644680944298


### Retrieve periodogram information

In [22]:
read_hdul['GLS_PG']

In [23]:
read_hdul['GLS_PG'].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   16 / length of dimension 1                          
NAXIS2  =              2856001 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    2 / number of table fields                         
TTYPE1  = 'freq    '                                                            
TFORM1  = 'D       '                                                            
TUNIT1  = 'd-1     '                                                            
TTYPE2  = 'power   '                                                            
TFORM2  = 'D       '        

In [24]:
read_hdul['GLS_PG'].data[:10]

FITS_rec([(2.4   , 0.12367587), (2.4001, 0.06539597),
          (2.4002, 0.01246361), (2.4003, 0.00383771),
          (2.4004, 0.03344694), (2.4005, 0.05916917),
          (2.4006, 0.05614809), (2.4007, 0.04069063),
          (2.4008, 0.0186047 ), (2.4009, 0.00912675)],
         dtype=(numpy.record, [('freq', '>f8'), ('power', '>f8')]))

In [25]:
read_pg_table = QTable.read(read_hdul['GLS_PG'])
read_pg_table

freq,power
1 / d,m2 / s2
float64,float64
2.4,0.12367587150689273
2.4001,0.06539597066744024
2.4002,0.012463611703876049
2.4003,0.0038377056360846457
2.4004,0.03344693872884501
2.4005,0.05916917457782217
2.4006,0.056148086954298335
2.4007,0.040690625560024396


In [26]:
read_pg_table['freq']

<Quantity [  2.4   ,   2.4001,   2.4002, ..., 287.9998, 287.9999, 288.    ] 1 / d>

In [27]:
diff_frecs = read_pg_table['freq'][:-1] - read_pg_table['freq'][1:]
diff_frecs

<Quantity [-1.e-04, -1.e-04, -1.e-04, ..., -1.e-04, -1.e-04, -1.e-04] 1 / d>

### Delete read information

In [28]:
read_hdul.close()

In [29]:
del read_hdul

## Summary of calculated periodograms

Number of objects with RV PG properly calculated:

In [30]:
gto[gto['valid_PG_RV'] == 1.0]

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_T0_PG_TESS,WF_e_T0_PG_TESS,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000997,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.689208,0.027575,999.999756,2.151008e-06,1.0,1.0,NaN,94.758838,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23548+38...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1353.926624,0.036481,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,J00183+440,GX And,A,15A,4.613226,44.024787,00:18:22.88,+44:01:22.6,116.690592,-18.446865,...,1764.644951,0.030860,999.999756,2.134945e-06,1.0,1.0,NaN,191.241441,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00183+44...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
386,J00162+198W,EZ Psc,Aab,1006A,4.064321,19.857041,00:16:14.63,+19:51:37.5,111.729451,-42.248173,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN
387,J00162+198E,LP 404-062,B,1006B,4.070651,19.860692,00:16:16.15,+19:51:50.5,111.738263,-42.245679,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN
388,J00067-075,GJ 1002,-,1002,1.676350,-7.546475,00:06:43.20,-07:32:17.0,92.444693,-67.730511,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN


There are three stars for which periodogram calculation failed. It was verified that in all these cases, the RV file is empty, with no data.

In [31]:
gto[gto['valid_PG_RV'] == 0.0][['Karmn', 'rv_file', 'valid_PG_RV', 'error_PG_RV']]

,Karmn,rv_file,valid_PG_RV,error_PG_RV
100,J17572+707,../data/CARM_VIS_RVs/J17572+707.avc.dat,0.0,zero-size array to reduction operation minimum...
126,J16102-193,../data/CARM_VIS_RVs/J16102-193.avc.dat,0.0,zero-size array to reduction operation minimum...
331,J04173+088,../data/CARM_VIS_RVs/J04173+088.avc.dat,0.0,zero-size array to reduction operation minimum...


# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- We have completed the basic GLS periodogram calculation for the radial velocity (RV) curves of CARM_VIS GTO objetcs (NZP-corrected), and stored the results.
- Out of the 390 initial objects:
  - 387 stars had their RV periodogram calculated correctly.
  - 3 of the objects yielded errors. In all three cases, the reason was that the RV file was empty, no RV points were present:
    - J17572+707
    - J16102-193
    - J04173+088